# Pruebas básicas
Una vez vistas las clases por las que está formada gymnasium y de que trata, se van a realizar pruebas con varios entornos que existen para ver el funcionamiento general de la librería. <br>

En esta primera prueba se va a lanzar un entorno de ejemplo con en el que las acciones que se van a tomar van a ser random, es decir, no se va a seguir ninguna política ni ningún tipo de entrenamiento. Este ejemplo, es únicamente para ver como sería una salida típica.

In [1]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset() #primer reset necesario

for _ in range(1000): #tiempo que está en ejecución el entorno
    action = env.action_space.sample() #accion random
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated: #finalizada la ejecución del anterior
        observation, info = env.reset()

env.close()

/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
